# Capstone Project 1: Data Wrangling

## Data Wrangling

The dataset downloaded from Lending Club was relatively clean aside from a high number of empty columns. I removed all but 17 of the columns from the original dataset. The columns removed were either empty, contained mostly null values or were not of interest to the current project.

### Import Packages and Dataset

In [1]:
# Import packages
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
# Import dataframe
df = pd.read_csv('LoanStats3a.csv', header = 1, dtype={'next_pymnt_d': object, 'id': object})
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Clean Dataframe

#### Drop columns and rows with no data 

Looking at the dataset, at least half of the columns seem to have all null values. I removed those columns using .dropna(). I also dropped the columns that only had 158 non-null values. This left me with 57 columns. 

In [3]:
# Drop all columns and rows with 0 non-null values 
df = df.dropna(axis = 1, how = 'all')
df = df.dropna(axis = 0, how = 'all')

In [4]:
# Drop columns with only 158 non-null values
df = df.iloc[:, 1:58]

### Drop additional columns

In order to focus my project, I decided to drop any columns that were not of interest to the project. This left me with 18 columns. 

I used the Lending Club data dictionary to help me decide which columns to remove from the dataframe. 
Data dictionary: https://resources.lendingclub.com/LCDataDictionary.xlsx

In [5]:
# Drop columns
df = df.iloc[:, 1:23]

In [6]:
# Drop additional columns
df = df.drop('funded_amnt_inv', 1)
df = df.drop('desc', 1)
df = df.drop('zip_code', 1)
df = df.drop('delinq_2yrs', 1)

#### Remove empty rows

I noticed that while most rows had 42535 non-null objects, there were 42538 entries in the dataframe. I used .isnull() to find and remove the 3 empty rows. 

In [7]:
# Identify the rows with missing data
nans = lambda df: df[df.isnull().all(axis=1)]
nans(df)

,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,addr_state,dti
39788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Drop empty rows 
df = df.drop([39788, 42540, 42541])

### Fill missing data
#### Employment Title

The employment title column is missing 2624 entries. I decided to replace those with 'Unknown'. I also replaced any values with a count of less than 20 to 'Other'.

In [9]:
# Replace NaN with 'Unknown'
df['emp_title'] = df['emp_title'].fillna('Unknown')

In [10]:
# Replace values of < 20 with 'Other'
df = df.assign(emp_title=df.groupby('emp_title')['emp_title'].transform(lambda x: x if x.size>=20 else 'Other'))

#### Annual Income

The Annual Income column is missing 4 values. I decided to use mean to fill those missing values

In [11]:
# Calculate the mean of annual_inc
inc_mean = df['annual_inc'].mean()
inc_mean

69136.55642025822

In [12]:
# Replace all the missing values in annual_inc with the mean
df['annual_inc'] = df['annual_inc'].fillna(inc_mean)

#### Title

The title column is missing 12 values. I decided to also replace these NaN values with 'Unknown'

In [13]:
## Replace NaN with 'Unknown'
df['title'] = df['title'].fillna('Unknown')

In [14]:
# Replace values of < 20 with 'Other'
df = df.assign(title=df.groupby('title')['title'].transform(lambda x: x if x.size>=20 else 'Other'))

### Additional Exploration

In order to see if any additional columns needed cleaning, I explored each column individually. 

#### Explore int_rate

In [15]:
# convert the percentages to a float
df['int_rate'] = df['int_rate'].str.rstrip('%').astype('float')/100.00

#### Explore installment

In [16]:
# convert installment to a float
df.installment = df.installment.astype('float')

#### Explore issue_d

In [17]:
df.issue_d.value_counts()

11-Dec    2267
11-Nov    2232
11-Oct    2118
11-Sep    2067
11-Aug    1934
11-Jul    1875
11-Jun    1835
11-May    1704
11-Apr    1563
11-Mar    1448
11-Jan    1380
10-Dec    1335
11-Feb    1298
10-Oct    1232
10-Nov    1224
10-Jul    1204
10-Sep    1189
10-Aug    1175
10-Jun    1105
10-May     989
10-Apr     912
10-Mar     828
10-Feb     682
9-Nov      662
10-Jan     662
9-Dec      658
9-Oct      604
9-Sep      507
9-Aug      446
9-Jul      411
9-Jun      406
8-Mar      402
9-May      359
9-Apr      333
9-Mar      324
8-Feb      306
8-Jan      305
9-Feb      302
9-Jan      269
8-Apr      259
8-Dec      253
8-Nov      209
7-Dec      172
8-Jul      141
8-Jun      124
8-Oct      122
8-May      115
7-Nov      112
7-Oct      105
8-Aug      100
7-Aug       74
7-Jul       63
8-Sep       57
7-Sep       53
7-Jun       24
Name: issue_d, dtype: int64

In [18]:
# Split issue_d into separate columns with year and month in each column
df[['issue_year','issue_month']] = df['issue_d'].str.split('-',expand=True)

# replace the incorrect years with the correct year format 
df['issue_year'] = df['issue_year'].replace({'7': '07',
                                               '8':'08',
                                            '9':'09'})

# combine the year and month columns
df.issue_d = df.issue_year.map(str) + '-' + df.issue_month

In [19]:
# Convert issue_d into datetime series
df.issue_d = pd.to_datetime(df.issue_d, format='%y-%b')

#### Explore loan_status

In [20]:
df.loan_status.value_counts()

Fully Paid                                             34116
Charged Off                                             5670
Does not meet the credit policy. Status:Fully Paid      1988
Does not meet the credit policy. Status:Charged Off      761
Name: loan_status, dtype: int64

In [21]:
# consolidate to two catagories
df['loan_status'] = df['loan_status'].replace({'Does not meet the credit policy. Status:Fully Paid':'Fully Paid',
                                               'Does not meet the credit policy. Status:Charged Off':'Charged Off'})
df.loan_status.value_counts()

Fully Paid     36104
Charged Off     6431
Name: loan_status, dtype: int64

In [22]:
# convert fully paid to 0 and charged off to 1
df['loan_status'] = df['loan_status'].replace({'Fully Paid':0,
                                               'Charged Off':1})
df.loan_status.value_counts()

0    36104
1     6431
Name: loan_status, dtype: int64

#### Explore pymnt_plan

In [23]:
df.pymnt_plan.value_counts()

n    42535
Name: pymnt_plan, dtype: int64

Since all columns have the same value, I removed this column. 

In [24]:
df = df.drop('pymnt_plan', 1)

### Results

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42535 entries, 0 to 42537
Data columns (total 19 columns):
funded_amnt            42535 non-null float64
term                   42535 non-null object
int_rate               42535 non-null float64
installment            42535 non-null float64
grade                  42535 non-null object
sub_grade              42535 non-null object
emp_title              42535 non-null object
emp_length             42535 non-null object
home_ownership         42535 non-null object
annual_inc             42535 non-null float64
verification_status    42535 non-null object
issue_d                42535 non-null datetime64[ns]
loan_status            42535 non-null int64
purpose                42535 non-null object
title                  42535 non-null object
addr_state             42535 non-null object
dti                    42535 non-null float64
issue_year             42535 non-null object
issue_month            42535 non-null object
dtypes: datetime64[ns](1

In [26]:
df.describe()

,funded_amnt,int_rate,installment,annual_inc,loan_status,dti
count,42535.000000,42535.000000,42535.000000,4.253500e+04,42535.000000,42535.000000
mean,10821.585753,0.121650,322.623063,6.913656e+04,0.151193,13.373043
std,7146.914675,0.037079,208.927216,6.409334e+04,0.358241,6.726315
min,500.000000,0.054200,15.670000,1.896000e+03,0.000000,0.000000
25%,5000.000000,0.096300,165.520000,4.000000e+04,0.000000,8.200000
50%,9600.000000,0.119900,277.690000,5.900000e+04,0.000000,13.470000
75%,15000.000000,0.147200,428.180000,8.250000e+04,0.000000,18.680000
max,35000.000000,0.245900,1305.190000,6.000000e+06,1.000000,29.990000


In [28]:
# Save the cleaned up dataset to a csv
df.to_csv('loan_clean.csv')